

Group nr:  48

Name 1 and CID: Soheb Kanon (CID) Soheb

Name 2 and CID: Othman Belal (CID) Othmanb

In [1]:
!pip install pygame
import numpy as np
import pygame
from IPython.display import Image
from BalanceRobot import BalanceRobot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 64.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
pygame 2.5.1 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Introduction to Jupyter Notebook

Jupyter Notebook is a web application that allows you to write code, text and equations in one file. Each code cell can be run individually, but often depend on variables defined in cells above it. This means when you first open the file you will need to run the code cells starting from the top.

The text boxes are using Markdown format and to write the equations it uses LaTex format which can be initialized with \\$ on each side of the equation, e.g. $ax + b = c$. To edit a text box you double-click them and then to get back the normal text you can run the text box. To run a cell either use the play button on the top or ctrl enter. 



## Balance robot

In this assignment, we will perform parameter estimation on a balancing robot. First, we will show how the equations of motion can be derived, some of the parameters in these equations will be unknown to us and linear regression will be used to estimate them. The linear regression method requires data to be collected, which will be collected while the robot is balancing with a control algorithm. Below we can see a drawing of the robot.


<img src="BallanceRobot.png" width="300"/>

| Parameter | Description | To be estimated |
| --- | --- | --- |
| $m_1$ | Mass of main body [kg]| Yes |
| $l$ | Distance from wheel hub to $m_1$ [m] | Yes |
| $I_1$ | Moment of inertia for main body [kg $\textrm{m}^2$]| Yes |
| $m_2$ | Mass of wheels [kg]| No |
| $r$ | Radious of wheels [m]| No |
| $I_2$ | Moment of inertia for of wheels [kg $\textrm{m}^2$]| No |

| State | Description | 
| --- | --- |
| $\alpha_1$ | Angle of main body [rad]| 
| $\alpha_2$ | Angle of wheel from position 0 [rad]|
| $x_1$ | Position of $m_1$ in x-axis for main body [m]|
| $y_1$ | Position of $m_1$ in y-axis for main body [m]|
| $x_2$ | Position of $m_2$ in x-axis for wheel [m]|



## Equations of motion

Here we show one way to calculate the equations of motion for the system. To do this we will use the Euler-Lagrange method which builds on finding the stationary solution between the kinetic and potential energy of the system. 

We start with setting up the kinematic relationships. We define $(x_1, y_1)$ to be the position for the centre of mass of the main body and $x_2$ the position of the wheels, both with respect to the coordinate frame. The kinematics equations for the system then become:

$x_2 = - \alpha_2 r$

$x_1 = - \alpha_2 r - l sin(\alpha_1)$

$y_1 = r + l cos(\alpha_1)$

$\dot{x}_1 = - r \dot{\alpha}_2 - l cos(\alpha_1) \dot{\alpha_1}$

$\dot{y}_1 = -l sin(\alpha_1) \dot{\alpha}_1$

$\dot{x}_2 = - r \dot{\alpha}_2 $ 

Potential Energy for the system:

$V = l cos(\alpha_1)m_1g$

Kinematic energy:

$T = \frac{1}{2} m_1 (\dot{x}^2_1 + \dot{y}^2_1) + \frac{1}{2} m_2 \dot{x}^2_2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2$

From these the equations of motion for the balance robot can be calculated, here we use the Euler Lagrange method. First the Lagrangian is calculated by: 

$L = T - V$

and becomes:

$L = \frac{1}{2} m_1 (r^2 \dot{\alpha}^2_2 + 2 r \dot{\alpha}_2 l cos(\alpha_1) \dot{\alpha_1} + l^2  \dot{\alpha}_1^2) +
\frac{1}{2} m_2 r^2 \dot{\alpha}_2^2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2 -l cos(\alpha_1)m_1g$

Euler Lagrange equation is defined by: 

$\frac{d}{dt}(\frac{\partial L }{\partial \dot{q}_i}) - \frac{\partial L }{\partial q_i} = Q_i$

Where $q_1 = \alpha_1$, $q_2 = \alpha_2$, and $Q_i$ represent external force, in our case, the external force is the torque $M$ generated by the motors between the main body and the wheels. 

The equations of motion after simplification becomes:

$M = m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + m_1 l^2 \ddot{\alpha}_1 + I_1 \ddot{\alpha}_1  - lsin(\alpha_1)m_1g$

and

$-M =  m_1 r^2 \ddot{\alpha}_2 + m_1 rlcos(\alpha_1) \ddot{\alpha}_1 - m_1 rlsin(\alpha_1)\dot{\alpha}_1^2 + m_2 r^2 \ddot{\alpha}_2 + I_2 \ddot{\alpha}_2$

## Exersise 1:

To perform parameter identification with linear regression the equations need to be reformulated to the form $ \mathbf{y} = \mathbf{X}^T \mathbf{\theta}$, where $\mathbf{y}$ is a vector with 2 elements, $\mathbf{X}$ is a matrix and the vector $\mathbf{\theta}$ will be estimated using regression. There are 3 parameters we want to estimate, $m_1$, $l$ and $I_1$, all other parameters can be assumed to be known. Rewrite the two equations of motion to the form $ y = X^T \theta $ so that $m_1$, $l$ and $I_1$ can be extracted from $\theta$. 

Hint: It might not be possible to linearize the equations so that $\theta$ directly has the parameters $m_1$, $l$, $I_1$ by themselves, i.e. $\theta = [m_1, l, I_1]^T$, but instead it might contain combinations of the parameters from which we later can calculate the values of $m_1$, $l$, $I_1$. 



Answer:

$\left[ \begin{array}{c}
 M\\
 -M-(m_2r^2+I_2)\ddot{\alpha}_2\\
\end{array} \right] = 
\left[ \begin{array}{ccc}
 0 &  r \ddot{\alpha}_2 l cos(\alpha_1) - gsin(\alpha_1)& \ddot{\alpha}_1 \\ 
    r^2\ddot{\alpha}_2  & rcos({\alpha}_1)\ddot{\alpha}_1 -rsin({\alpha}_1)\dot{\alpha}_1^2  & 0 \end{array} \right]
\left[ \begin{array}{c}
  m_1\\
  m_1l\\
  m_1l^2+I_1\\
\end{array} \right]$




## Simulator

For those interested, the equations of motion are not only interesting for parameter identification but can also be used to build a simulator for the system. The equations of motion are rewritten to a form where the state of the robot is easier to integrate.

$ \ddot{\alpha}_2   = \frac{-M -m_1 rlcos(\alpha_1) (M + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1) + m_1 rlsin(\alpha_1)\dot{\alpha}_1^2}{(m_1 r^2  - m_1^2 r^2 l^2 cos^2(\alpha_1)/ (m_1 l^2 + I_1) + m_2 r^2 + I_2)} $

$\ddot{\alpha}_1  = (M -  m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1)$

From these equations, we can simulate the robot by either integrating accelerations with a fixed time step to get the velocity and positions or we can use a differential solver e.g. runge-kutta 45.  

## Controller
The robot is essentially an inverted pendilum on wheels, without a controller it will fall. The controller which can be seen below keeps the robot upright and moves it towords a referrence position on the x-axis. In the update function, there is a $\textit{pos_ref}$ variable that determines the reference signal for the controller, and will later be modified.  

<img src="Controller.png" width="500"/>

In [2]:
class Controller(object):
    def __init__(self, dt, init_states):
        self.dt = dt
        # controller parameter for position controller
        self.K_pos = 3
        self.D_pos = 0.5
        # controller parameter for angle controller
        self.K_ang = 10
        self.I_ang = 1
        self.D_ang = 1
        # limit the maximum allowed reference angle
        self.max_angle = 0.8
        # variables for the integrator in the PID controller
        self.angle_int = 0
    
    def update(self, t, states): 
        # The update function will be called on every timestep and return the torque for the motors. 
        
        # The position reference to follow
        # ----------------------------------------------------------------
        pos_ref =  0.2*np.sin(t*2)
        # ----------------------------------------------------------------
        
        # PD controller for position
        err = self.K_pos*(pos_ref - states['pos']) - self.D_pos * states['vel']
        # The error is used as a reference angle, with upper and lower bounds. 
        ref_ang = -np.clip(err, -self.max_angle, self.max_angle)
        self.angle_int += self.dt * (ref_ang - states['a1'])
        # PID controller for the angle, generates the desired motor torque. 
        M = self.K_ang*(ref_ang-states['a1']) + self.I_ang*self.angle_int - self.D_ang*states['da1']
    
        return M


## Data collection

We need to collect data before we can do any regression on the problem. The data is collected when the robot is following a position reference signal. In the code below the data is collected in vectors, e.g. $\textrm{da1} = [\dot{\alpha}_1(t_0), \dot{\alpha}_1(t_1), ... , \dot{\alpha}_1(t_n)]$ and so on. The known parameters are g, r, m2 and I2. 

There are two variables you can change:

scale_noise: which scales the added noise to the measurements. 

sim_speed: which allows you to run the simulation faster then realtime. 


In [3]:
import pygame
import numpy as np

class Render(object):
    def __init__(self, caption, resolution=(1000, 1000)):
        self.window = pygame.display.set_mode(resolution)
        pygame.display.set_caption(caption)
        self.w = resolution[0]
        self.h = resolution[1]
        self.render_layers = dict()
        self.clock = pygame.time.Clock()

    def add_object(self, key, layer, obj):
        # check if layer exists in dict, layer and key should be strings.
        if layer not in self.render_layers:
            self.render_layers[layer] = dict()
        self.render_layers[layer][key] = obj

    def remove_object(self, key):
        for i, (layer, value) in enumerate(self.render_layers.items()):
            if key in value:
                self.render_layers[layer].pop(key, None)
                return

    def check_key_exists(self, key, layer=None):
        if layer is None:
            for i, (layer, value) in enumerate(self.render_layers):
                if key in value:
                    return True
        else:
            for v in self.render_layers[layer]:
                if key in v:
                    return True
        return False

    def get_layer(self, layer):
        if layer not in self.render_layers:
            return dict()
        return self.render_layers[layer]

    def get_object(self, key):
        for i, (layer, value) in enumerate(self.render_layers.items()):
            if key in value:
                return self.render_layers[layer][key], layer
        return None, None

    def update_object(self, obj, key, layer=None):
        if layer is None:
            for i, (layer, value) in enumerate(self.render_layers.items()):
                if key in value:
                    self.render_layers[layer][key] = obj
        self.render_layers[layer][key] = obj

    def update_window(self, fps=None):
        self.window.fill((100, 100, 100))
        for i, (layer, value) in enumerate(self.render_layers.items()):
            for j, (key, obj) in enumerate(value.items()):
                self.window.blit(obj.get_rot_surface(), obj.get_pos())
        pygame.display.update()
        if fps is not None:
            self.clock.tick(fps)

    def get_resolution(self):
        return self.w, self.h

    def get_main_window(self):
        return self.window


class RenderObject(object):
    def __init__(self, surf, x, y):
        '''
        :param surf:
        :param x: Horizontal (top left (0,0))
        :param y: Vertical
        '''
        self.surf = surf
        self.rot_surf = surf
        self.x = x
        self.y = y
        self.pos = self.surf.get_rect(center=(x, y))
        self.angle = 0

    def set_angle(self, angle):
        self.angle = 180*angle/np.pi

    def get_rot_surface(self):
        self.rot_surf = pygame.transform.rotate(self.surf, self.angle)
        return self.rot_surf

    def get_surface(self):
        return self.surf

    def update_surface(self, surface):
        self.surf = surface

    def get_pos(self):
        return self.rot_surf.get_rect(center=(self.x, self.y))

    def get_pos_xy(self):
        return self.x, self.y

    def set_pos(self, x, y):
        self.x = x
        self.y = y


ModuleNotFoundError: No module named 'pygame'

In [4]:
!pip install pygame==2.5.1


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
from scipy.integrate import solve_ivp
import pygame_render
import pygame

def flatten(t):
    return [item for sublist in t for item in sublist]


class BalanceRobot():
    def __init__(self, dt, sim_speed=1):
        self.p = {'r': 0.05,
                  'l': 0.15,
                  'm1': 0.5,
                  'm2': 0.1,
                  'g': 9.81,
                  'I1': 0.006,
                  'I2': 0.002}

        self.canvas_w = 1
        self.canvas_h = 1
        self.step_time = dt
        self.y0 = [0, 0.001, 0, 0]
        self.render = pygame_render.Render(caption='Balance robot')
        self.w, self.h = self.render.get_resolution()
        self.frame_rate = 30
        self.fac = self.frame_rate*self.step_time/sim_speed

        self.frame_nr = 0
        w_px = int(0.08*self.w*self.canvas_w)
        l_px = int(2*self.p['l']*self.h*self.canvas_h)
        self.r_px = int(self.p['r'] * self.h * self.canvas_h)
        b_h = 800
        b_w = 800

        surf = pygame.Surface((b_w, b_h), pygame.SRCALPHA)
        points = [((b_w-w_px)//2, b_w//2), ((b_w-w_px)//2, (b_h-l_px)//2),
                  ((b_w+w_px)//2, (b_h-l_px)//2), ((b_w+w_px)//2, b_w//2)]
        _ = pygame.draw.polygon(surface=surf, color=(200, 200, 200, 150), points=points)
        self.body = pygame_render.RenderObject(surf=surf, x=self.w//2, y=self.h-self.r_px)

        self.render.add_object(key='body', layer='1', obj=self.body)

        surf2 = pygame.Surface((b_w, b_h), pygame.SRCALPHA)
        _ = pygame.draw.circle(surface=surf2, color=(255, 150, 150, 150), center=(b_w//2, b_h//2),
                               radius=int(self.p['r']*self.h))
        _ = pygame.draw.line(surface=surf2, color=(0, 0, 0, 150), start_pos=(b_w//2-int(self.p['r']*self.h), b_h//2),
                             end_pos=(b_w // 2 + int(self.p['r']*self.h), b_h // 2), width=3)

        self.wheel = pygame_render.RenderObject(surf=surf2, x=self.w//2, y=self.h-self.r_px)
        self.render.add_object(key='wheel', layer='2', obj=self.wheel)


    def step(self, torque):
        disturbance = np.random.normal(0, 0.0)
        dx = self.balance_sim(0, self.y0, torque)

        sol = solve_ivp(fun=self.balance_sim, t_span=[0, self.step_time], y0=self.y0, method='RK45',
                        args=[torque],
                        dense_output=False, t_eval=[self.step_time])


        self.y0 = flatten(sol.y)

        # for rendering only
        self.body.set_angle(self.y0[1])
        x = -int(self.y0[3] * self.p['r'] * self.w)
        self.body.set_pos(x=self.w//2 + x, y=self.h-self.r_px)
        self.wheel.set_angle(self.y0[3])
        self.wheel.set_pos(x=self.w//2 + x, y=self.h-self.r_px)
        if (self.frame_nr*self.fac)%1 > (1-self.fac/2) or (self.frame_nr*self.fac)%1 < (self.fac/2):
            self.render.update_window(fps=self.frame_rate)
        self.frame_nr += 1
        return dx[0], dx[2]

    def get_state(self):
        pos = -self.p['r'] * self.y0[3] - self.p['l'] * np.sin(self.y0[1])
        vel = -self.p['r'] * self.y0[2] - self.p['l'] * np.cos(self.y0[1]) * self.y0[0]
        states = {'da1': self.y0[0],
                  'a1': self.y0[1],
                  'da2': self.y0[2],
                  'a2': self.y0[3],
                  'vel': vel,
                  'pos': pos}
        return states

    def balance_sim(self, t, X, M):
        # X = [do1, o1, do2, o2]
        do1 = X[0]
        o1 = X[1]
        do2 = X[2]
        o2 = X[3]
        r = self.p['r']
        l = self.p['l']
        m1 = self.p['m1']
        m2 = self.p['m2']
        g = self.p['g']
        I1 = self.p['I1']
        I2 = self.p['I2']

        ddo2 = (-M - (m1*r*l*np.cos(o1) * (M + l*m1*g*np.sin(o1))/(m1*(l**2)+I1)) + m1*r*l*np.sin(o1)*(do1**2)) / \
               (m1*r**2 - ((r*l*m1*np.cos(o1))**2)/(m1*l**2 + I1) + m2*r**2 + I2)

        ddo1 = (M - m1*r*ddo2*l*np.cos(o1) + m1*g*l*np.sin(o1))/(m1*l**2 + I1)
        dX = [ddo1, do1, ddo2, do2]
        return dX




pygame 2.5.1 (SDL 2.28.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
## scale_noise, keep at 1 until later, change it later in exercise 7. 
# It controls the amount of noise applied to the measurements

# ----------------------------------------------------------------
scale_noise = 0.5
# ----------------------------------------------------------------

## Simulation speed, increase it for faster than real-time simulation.
sim_speed = 1

# number of steps and step size to run the simulation/data collection
n_steps = 1000
step_time = 1e-2

# Initiate the robot
robot = BalanceRobot(dt=step_time, sim_speed=sim_speed)
try:
    # True parameters in the robot.
    g = robot.p["g"] # gravity.
    r = robot.p["r"] # radius of wheel.
    m2 = robot.p["m2"] # mass of wheel.
    I2 = robot.p["I2"] # moment of intertia of wheel.

    # Initate the controller 
    state = robot.get_state()
    controller = Controller(dt=step_time, init_states=state)

    da1 = np.zeros(n_steps)
    a1 = np.zeros(n_steps)
    da2 = np.zeros(n_steps)
    a2 = np.zeros(n_steps)
    dda1 = np.zeros(n_steps)
    dda2 = np.zeros(n_steps)
    M = np.zeros(n_steps)

    ## Main loop 

    for i in range(n_steps):
        # calculate time
        t = i*step_time
        # calculate torque 
        state = robot.get_state()
        M_ = controller.update(t, state)
        # step one time step for robot simulation
        dda1_, dda2_ = robot.step(torque=M_)

        ## Add noice to data
        da1[i] = state['da1'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 1
        a1[i] = state['a1'] + np.random.normal(0.0, scale_noise*0.03) # alpha 1
        da2[i] = state['da2'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 2
        a2[i] = state['a2'] + np.random.normal(0.0, scale_noise*0.03) # alpha 2
        dda1[i] = dda1_ + np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 1
        dda2[i] = dda2_ + np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 2
        M[i] = M_ + np.random.normal(0.0, scale_noise*0.05) # torque 

    pygame.quit()
except:
    pygame.quit()


NameError: name 'BalanceRobot' is not defined

For the first test, we set the position reference signal to be a sinusoidal shape, e.g. $\textit{pos_ref} = 0.2 \sin(2t)$. What would we expect to happen if we used a constant reference signal, i.e. $\textit{pos_ref} = 0$, instead? Which of the two reference signal would we expect to give better parameter estimation and why? 



Answer: 
When we use a constant value e.g 0, 1 , 2, 3 we get the system to act like a statik system in which makes the equations to be unable to give an accurate estimation for the values we seek to estimate. On the other hand using the position reference signal such as 0.2sin(2t) makes the system a dynamic system which is a good condition for our equation of motion in order to get much more accurate estimations. 



## Exercise 3: Linear regression 

Use linear regression to estimate the parameters $m_1$, $l$ and $I_1$. You have access to the vectors da1, da, da2, a2, dda1, dda2 and M, and you can use the variables g, r, m2 and I2.    

In [7]:
sum_mat_x_nul_xtran = np.zeros((3, 3))
sum_mat_x_nul_y = np.zeros(3)

# frågan är ska man använda cross multiplication (np.mul) eller dot production (np.dot)


for i in range(n_steps):
    matrix_x = np.array([[0, 0.0025 * dda2[i]],
                         [0.05 * dda2[i] * np.cos(a1[i]) - 9.81 * np.sin(a1[i]), 0.05 * dda1[i] * np.cos(a1[i]) - 0.05 * np.sin(a1[i]) * (da1[i] ** 2)],
                         [dda1[i], 0]])
    matrix_y = np.array([M[i], -M[i]-0.00225 * dda2[i]])# jag måste fixa det
    
    
    matrix_tran = np.transpose(matrix_x)
    matrix_x_mul = np.dot(matrix_x,matrix_tran)
    sum_mat_x_nul_xtran = sum_mat_x_nul_xtran + matrix_x_mul
    
    matrix_xy_mul = np.dot(matrix_x,matrix_y)
    sum_mat_x_nul_y = sum_mat_x_nul_y + matrix_xy_mul
    

last_x = np.linalg.inv(sum_mat_x_nul_xtran)

theta_hat = np.dot(last_x,sum_mat_x_nul_y)

# man måste använda dot multiplication 
# man ska ha tre variablier i theta 
# man när man använder error **2 so det man måste ta transponat sen skalar prodcut 

ValueError: operands could not be broadcast together with shapes (4,4) (3,3) 

In [8]:
print(theta_hat)

[0.47242136 0.06757776 0.01840679]


In [9]:
#Answer: 

x = theta_hat[0]
y = theta_hat[1]
z = theta_hat[2]


est_m1 = x # 0.449
est_l = y/x # 0.132
est_I1 = z - ((y**2)/x) # 0.011

print('Estimated mass = ', np.round(est_m1, 3), ' and true  = ', robot.p['m1'])
print('Estimated length = ', np.round(est_l, 3), ' and true value = ', robot.p['l'])
print('Estimated moment of inertia = ', np.round(est_I1, 3), ' and true value = ', robot.p['I1'])



Estimated mass =  0.472  and true  =  0.5
Estimated length =  0.143  and true value =  0.15
Estimated moment of inertia =  0.009  and true value =  0.006


## Exercise 4: Calculate the cost function 

When solving the linear regression problem we minimize a cost function $J(\theta) = \frac{1}{2 n} \sum_{i=1}^n \varepsilon_i^2$. Calculate the value of the cost function on the saved data with the parameters estimated. 

Hint: n = n_steps.

In [10]:
co_val = 0
# samma frågan här 
for i in range(n_steps):
    matrix_x = np.array([[0, 0.0025 * dda2[i]],
                         [0.05 * dda2[i] * np.cos(a1[i]) - 9.81 * np.sin(a1[i]), 0.05 * dda1[i] * np.cos(a1[i]) - 0.05 * np.sin(a1[i]) * (da1[i] ** 2)],
                         [dda1[i], 0]])
    matrix_y = np.array([M[i], -M[i]-0.00225 * dda2[i]])
    
    theta_trans = np.transpose(theta_hat)
    thetatra_mul_x = np.dot(theta_trans,matrix_x)# kolla om det är matmul eller dot
    error = matrix_y - thetatra_mul_x
    error_trans = np.transpose(error)
    errr_uptill2 = np.dot(error_trans,error)# error**2 #np.dot(error,error)
    co_val = co_val + errr_uptill2
    
cost_value = co_val/n_steps

In [11]:
#Answer: 

# cost_value = 0.005403973775531635
# 0.0046975696794038695
print('cost_value', cost_value)

cost_value 0.0013367796260538034


## Exercise 5: Reference signal

Try to change the position reference signal in the controller, try a few different ones and report the identified parameters and loss value below together with the function used. You only need to change pos_ref in the update function in the controller. 

Hint: Try more complex functions by stacking sinusoidal, e.g. $\textit{pos_ref} = 0.2sin(2t) + 0.03cos(7t)$.


Answer:

| ref signal | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0 | 0.0046975696794038695 | 1.334 | 0 | 0.014 | 
| 0.2*np.sin(t*2) | 0.005403973775531635 | 0.449 | 0.132 | 0.011  | 
| 0.2sin(2t) + 0.03cos(7t) | 0.005555922628695368 | 0.475 | 0.141 | 0.007 | 
|0.2sin(2t) + 0.03cos(7t) +0.5sin(3t) + 0.08cos(4t) | 0.005781134771940516 | 0.501| 0.15 | 0.006 | 

## Exercise 6: Analyse cost function and reference signals. 

1. How do more complex reference signals affect parameter estimation?
2. Does the cost function value improve with better-identified parameters, why/why not?
3. Could you keep making the reference signal more and more complex for better estimation or is there a limit, why?


Answer:

1 - When we impliment a much more complex reference signal we get the system to be much more dynamic and therefore we get to estimate the parameters much more accurate than a simple reference signal. 

2- To get more accurate parameters we used much more complix reference position signal. That leads to get a higher cost function value. 


3- No we can only improve the estimation accuracy of our parameters up to a certain point. There is a limit to how complix reference signal can be helpfull. The complix signal should be related to the computational resources we have. If we exceed the limits of our algorithm and computational resources the limit may lead to unaccurate results and estimations. That is why it is so important to think about the complix reference signal and its trade-off with the estimation accuracy




 ## Exercise 7:  Noise

Explore how the noise levels added before the data collection affects the identification results and also the value of the cost function. Why is it that the cost function reacts differently compared to exercise 5? Use the position reference signal with the best parameter identification from exercise 5.

Answer: 
The higher the scale noise is gives us a much more larger variance in our system which leads to an unaccurate estimation of our cost function value. 


$\textit{pos_ref} = $

| scale_noise | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0.1 | 5.193728528232166e-05   | 0.501  | 0.149 | 0.006  | 
| 0.5 | 0.0014054455448360413   | 0.499  |0.136  | 0.01   | 
| 1   | 0.005453426231777198    | 0.453  | 0.128 | 0.012  | 
| 2   | 0.0046975696794038695   | 0.46   | 0.064 | 0.023  | 
| 10  |  0.500766498937574      | 0.441  | 0.004 | 0.031  | 


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=568610fc-9ec2-4154-9999-7e2bc7592f65' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>